In [13]:
import os
import h5py
import numpy as np
from torchdeepretina.datas import loadexpt, _loadexpt_h5

In [14]:
data_path = os.path.join('/home/salamander/experiments/data/')

In [5]:
data = h5py.File(data_path, mode='r')

In [9]:
data.keys()

<KeysViewHDF5 ['spikes', 'test', 'train']>

In [11]:
data['spikes'].keys()

<KeysViewHDF5 ['cell01', 'cell02', 'cell03', 'cell04', 'cell05', 'cell06', 'cell07', 'cell08', 'cell09']>

In [14]:
data['train'].keys()

<KeysViewHDF5 ['response', 'stimulus', 'time']>

In [18]:
np.asarray(data['train']['stimulus']).shape

(359802, 50, 50)

In [20]:
data['train']['response'].keys()

<KeysViewHDF5 ['binned', 'firing_rate_10ms', 'firing_rate_20ms', 'firing_rate_5ms']>

In [21]:
np.asarray(data['train']['response']['binned']).shape

(9, 359802)

In [29]:
np.asarray(data['train']['response']['firing_rate_10ms']).shape

(9, 359802)

In [47]:
print(np.asarray(data['train']['response']['firing_rate_10ms'])[0,6])

0.47261764030129316


In [48]:
print(np.asarray(data['train']['response']['binned'])[0,6])

0.0


In [3]:
import torch
from collections import deque

In [4]:
model = KineticsModel()


In [5]:
def get_hs(model, batch_size, device):
    hs = []
    hs.append(torch.zeros(batch_size, *model.h_shapes[0]).to(device))
    hs[0][:,0] = 1
    hs.append(deque([],maxlen=model.seq_len))
    for i in range(model.seq_len):
        hs[1].append(torch.zeros(batch_size, *model.h_shapes[1]).to(device))
    return hs

In [4]:
hs = get_hs(model, 5)

In [7]:
batch_size = 5
x = torch.ones(batch_size, 40, 50, 50)

In [10]:
out, hs = model(x, hs)

In [11]:
out.shape

torch.Size([5, 5])

In [21]:
from torch.utils.data.sampler import Sampler
class BatchRnnSampler(Sampler):
    
    def __init__(self, length, batch_size, seq_len):
        self.length = length
        self.batch_size = batch_size
        self.seq_len = seq_len

    def __iter__(self):
        batch_idx = 0
        count = 0
        while batch_idx < self.length // self.batch_size:
            batch = [batch_idx + n * self.__len__() for n in range(self.batch_size)]
            yield batch
            batch_idx += 1
            count += 1
            if count == self.seq_len:
                count = 0
                batch_idx -= (self.seq_len - 1)

    def __len__(self):
        return (self.length // self.batch_size - self.seq_len + 1) * self.seq_len + self.seq_len -1

In [23]:
len(BatchRnnSampler(length=30, batch_size=3, seq_len=5))

34

In [20]:
len(BatchRnnSampler(length=30, batch_size=3, seq_len=5))

10

In [23]:
data = loadexpt('15-10-07', [0,1,2,3,4], 'naturalscene', 'train', 40, 0, data_path=data_path)

In [24]:
data.X.shape

(359762, 40, 50, 50)

In [26]:
data.y.shape

(359762, 5)

In [15]:
from kinetic.data import BatchRnnSampler, TrainDataset, ValidationDataset
dataset = TrainDataset(data_path)

In [16]:
from torch.utils.data.dataloader import DataLoader
batch_sampler = BatchRnnSampler(length=len(dataset), batch_size=512, seq_len=8)
data = DataLoader(dataset=dataset, batch_sampler=batch_sampler)

In [19]:
len(batch_sampler)

644

In [8]:
for i,(x,y) in enumerate(data):
    if i == 0:
        inpt = x
        trgt = y
        break

In [9]:
inpt.shape

torch.Size([512, 40, 50, 50])

In [ ]:
dataset

In [8]:
from scipy.stats import pearsonr
device = torch.device('cuda:1')
hs = get_hs(model, 1, device)
model = model.to(device)
model.eval()
with torch.no_grad():
    pearsons = []
    val_pred = []
    val_targ = []
    for x,y in data:
        x = x.to(device)
        out, hs = model(x, hs)
        val_pred.append(out.detach().cpu().numpy().squeeze())
        val_targ.append(y.detach().numpy().squeeze())
    val_pred = np.stack(val_pred, axis=0)
    val_targ = np.stack(val_targ, axis=0)
    for cell in range(5):
        pearsons.append(pearsonr(val_pred[:,cell],val_targ[:,cell])[0])
    print(np.array(pearsons).mean())

-0.0035025242913605073


In [10]:
val_targ.shape

(30000, 5)

In [3]:
data = loadexpt('15-10-07', [0,1,2,3,4], 'naturalscene', 'train',
                40, 0, data_path=data_path)

In [18]:
X = data.X
batch_size = 512
length = X.shape[0]//batch_size
tot_len = length*batch_size
X_trunc = X[:tot_len].reshape(batch_size, length, *X.shape[1:])
X_trunc = np.transpose(X_trunc, (1,0,2,3,4))
y = data.y
batch_size = 512
length = y.shape[0]//batch_size
tot_len = length*batch_size
y_trunc = y[:tot_len].reshape(batch_size, length, *y.shape[1:])
y_trunc = np.transpose(y_trunc, (1,0,2))

In [19]:
y_trunc.shape

(702, 512, 5)

In [4]:
device = torch.device('cuda:1')

In [5]:
print(device)

cuda:1


In [4]:
import torch.nn as nn
import torch
nn.Parameter(torch.ones((5,2)).float(), requires_grad=True)

Parameter containing:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)